### Part 0:
- vmd command:
   - segid SUBA
   - segid SUBA and resid 50
- ![](../images/pic1.png)

- vmd command:
   - resid 1 or (resid 2 and name C1)
- ![](../images/pic2.png)

In [1]:
from os import path
import numpy as np
from glycantool import PDB
rootfolder = '/home/yizaochen/codes/glycantest'

### Part 1-0: Read Protein PDB

In [2]:
pdb_in = path.join(rootfolder, 'glycan', 'mers_a_5x59.pdb')
mers_a = PDB.PDBReader(pdb_in, skip_header=1, skip_footer=1, segid_exist=True)

### Part 1-1: Get ND2 and HD21

In [3]:
atom_wanted = dict()
for atom in mers_a.atomgroup:
    if (atom.resid == 50) & (atom.resname == 'ASN') & (atom.name == 'ND2'):
        atom_wanted['ND2'] = atom
    elif (atom.resid == 50) & (atom.resname == 'ASN') & (atom.name == 'HD21'):
        atom_wanted['HD21'] = atom

### Part 1-2: Define $\vec{v}_1 = \vec{r}_{\mathrm{HD21}} - \vec{r}_{\mathrm{ND2}}$

In [4]:
v1 = np.zeros(3)
v1[0] = atom_wanted['HD21'].x - atom_wanted['ND2'].x
v1[1] = atom_wanted['HD21'].y - atom_wanted['ND2'].y
v1[2] = atom_wanted['HD21'].z - atom_wanted['ND2'].z

### Part 2-0: Read Sugar PDB
- `cp man9.pdb man9.pdbreader.pdb`
- `vim man9.pdbreader.pdb` delete TER

In [5]:
pdb_in = path.join(rootfolder, 'glycan', 'man9.pdbreader.pdb')
m9 = PDB.PDBReader(pdb_in, skip_header=1, skip_footer=1, segid_exist=False)

### Part 2-1: Get O1 and C1

In [6]:
atom_wanted_m9 = dict()
for atom in m9.atomgroup:
    if (atom.resid == 1) & (atom.resname == 'ROH') & (atom.name == 'O1'):
        atom_wanted_m9['O1'] = atom
    elif (atom.resid == 2) & (atom.resname == '4YB') & (atom.name == 'C1'):
        atom_wanted_m9['C1'] = atom

In [7]:
atom_wanted_m9

{'O1': x: -5.484  y: 5.892  z: 1.199, 'C1': x: -5.010  y: 4.588  z: 0.853}

### Part 2-2: Define $\vec{v}_2 = \vec{r}_{\mathrm{C1}} - \vec{r}_{\mathrm{O1}}$

In [8]:
v2 = np.zeros(3)
v2[0] = atom_wanted_m9['C1'].x - atom_wanted_m9['O1'].x
v2[1] = atom_wanted_m9['C1'].y - atom_wanted_m9['O1'].y
v2[2] = atom_wanted_m9['C1'].z - atom_wanted_m9['O1'].z

### Part 3: Translation sugar: Move O1's position to ND2's position
### Define $\vec{t}_1 = \vec{r}_{\mathrm{ND2}} - \vec{r}_{\mathrm{O1}}$

In [9]:
t1 = np.zeros(3)
t1[0] = atom_wanted['ND2'].x - atom_wanted_m9['O1'].x
t1[1] = atom_wanted['ND2'].y - atom_wanted_m9['O1'].y
t1[2] = atom_wanted['ND2'].z - atom_wanted_m9['O1'].z

### do translation and output PDB

In [11]:
atomgroup_after_translation = list()
for atom in m9.atomgroup:
    atom.x += t1[0]
    atom.y += t1[1]
    atom.z += t1[2]
    atomgroup_after_translation.append(atom)

In [12]:
pdb_out = path.join(rootfolder, 'glycan', 'man9.after_translation.pdb')
m9_after_translation = PDB.PDBWriter(pdb_out, atomgroup_after_translation)
m9_after_translation.write_pdb()

Write PDB: /home/yizaochen/codes/glycantest/glycan/man9.after_translation.pdb


### Merge

In [16]:
pdb1 = path.join(rootfolder, 'glycan', 'mers_a_5x59.pdb')
pdb2 = path.join(rootfolder, 'glycan', 'mers_a_5x59.merge_trans.pdb')
pdb3 = path.join(rootfolder, 'glycan', 'man9.after_translation.pdb')
print(f'cp {pdb1} {pdb2}')
print(f'vim {pdb2}')
print(f':read {pdb3}')
print('Use vim to add M9 at the end of Sugar')
print('Then, delete target atoms')

cp /home/yizaochen/codes/glycantest/glycan/mers_a_5x59.pdb /home/yizaochen/codes/glycantest/glycan/mers_a_5x59.merge_trans.pdb
vim /home/yizaochen/codes/glycantest/glycan/mers_a_5x59.merge_trans.pdb
:read /home/yizaochen/codes/glycantest/glycan/man9.after_translation.pdb
Use vim to add M9 at the end of Sugar
Then, delete target atoms


### Use VMD to check

In [17]:
print('vmd -pdb mers_a_5x59.merge_trans.pdb')
print('segid SUBA')

vmd -pdb mers_a_5x59.merge_trans.pdb
segid SUBA
